In [15]:
#
import json
import os
import matplotlib.pyplot as plt
import numpy as np
import torch as t
from torch import nn
import torch.optim as optim
from torch.utils.data import dataloader
import torchvision
from torchvision import models, datasets , transforms
import imageio
import time
import warnings
import random
import sys
import copy
import json
from PIL import Image

In [ ]:
data_dir = "E:\\download\\102flowers"
# 数据处理
## 数据增强dataAugmentation
data_transforms = {
    "train":
    transforms.Compose([transforms.RandomRotation(degrees = (-45,45)),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(p = 0.5),
    transforms.RandomVerticalFlip(p = 0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.1, saturation=0.1, hue=0.2),
    transforms.RandomGrayscale(p=0.02),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    #transforms.Resize((224, 224))
    ]),
    "valid":
        transforms.Compose([transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

batch_size =8
image_datasets = {x:datasets.ImageFolder(os.path.join(data_dir,x),data_transforms[x]) for x in ['train','valid']}
print(image_datasets)
dataloaders = {x:t.utils.data.DataLoader(image_datasets[x],batch_size=batch_size,shuffle=True) for x in ['train','valid']}
datasets_sizes = {x:len(image_datasets[x]) for x in ['train','valid']}
class_names = image_datasets['train'].classes


{'train': Dataset ImageFolder
    Number of datapoints: 6587
    Root location: E:\download\102flowers\train
    StandardTransform
Transform: Compose(
               RandomRotation(degrees=[-45.0, 45.0], interpolation=nearest, expand=False, fill=0)
               CenterCrop(size=(224, 224))
               RandomHorizontalFlip(p=0.5)
               RandomVerticalFlip(p=0.5)
               ColorJitter(brightness=(0.8, 1.2), contrast=(0.9, 1.1), saturation=(0.9, 1.1), hue=(-0.2, 0.2))
               RandomGrayscale(p=0.02)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           ), 'valid': Dataset ImageFolder
    Number of datapoints: 1602
    Root location: E:\download\102flowers\valid
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=bilinear, max_size=None, antialias=True)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.4

### 直接加载RESNET模型进行迁移学习

In [9]:
model_name = 'resnet152'
feature_extractor = True

In [6]:
#检查gpu
train_on_gpu = t.cuda.is_available()
if train_on_gpu:
    print('CUDA is available!  Training on GPU ...')
else:
    print('CUDA is not available.  Training on CPU ...')
device = t.device("cuda" if train_on_gpu else "cpu")

CUDA is available!  Training on GPU ...


In [4]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

#### 查看resnet结构

In [8]:
model_ft = models.resnet152()
#model_ft

In [ ]:
def initalize_model(model_name,class_num,feature_extractor,use_pretrained = True):#初始化模型(model_name:模型名称,class_num:类别数量,feature_extractor:是否冻结特征提取层,use_pretrained:是否使用预训练模型)
    if model_name == "resnet152":
        model_ft = models.resnet152(weights=models.ResNet152_Weights.IMAGENET1K_V1)
        set_parameter_requires_grad(model_ft, feature_extractor)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Sequential(
            nn.Linear(num_ftrs, class_num),
            # nn.ReLU(),
            # nn.Dropout(0.2),
            # nn.Linear(256, class_num),
            nn.LogSoftmax(dim=1)
        )
        input_size = 224
    return model_ft,input_size


In [ ]:
model_ft,input_size = initalize_model(model_name,102,feature_extractor,use_pretrained = True)
model_ft = model_ft.to(device)#更改训练设备
#用于保存网络
filename = 'my_checkpoint.pth'

params_to_update = model_ft.parameters()
if feature_extractor:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print(name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print(name)

fc.0.weight
fc.0.bias


#### 查看修改后的网络架构

In [22]:
#model_ft

#### 优化器设置

In [ ]:
optimizer = optim.Adam(param)

TypeError: Adam.__init__() missing 1 required positional argument: 'params'